# Time Series Analysis - Tracking with MLFLow

## 1. Install and Import MLFlow and Ngrok

### 1.  Import & Setup MLFLow

In [1]:
import mlflow

In [2]:
# Define the MLflow storage path on local Drive
mlflow_storage_path = "/Users/lukasfichtner/Documents/Guayas_project_week3/mlflow_results"
# Set MLFlow  to log to zhe local drive directory
mlflow.set_tracking_uri(f"file:{mlflow_storage_path}")

In [3]:
# Set up experiment name
mlflow.set_experiment("XGB")  

<Experiment: artifact_location='file:///Users/lukasfichtner/Documents/Guayas_project_week3/mlflow_results/464531910681999232', creation_time=1756727380633, experiment_id='464531910681999232', last_update_time=1756727380633, lifecycle_stage='active', name='XGB', tags={}>

In [4]:
from pyngrok import ngrok, conf
import getpass
import subprocess
import os

In [ ]:
# Zuerst alle Prozesse auf Port 5000 beenden
subprocess.run(["pkill", "-f", "mlflow"])  # MLflow Prozesse killen
subprocess.run(["lsof", "-ti:5000", "|", "xargs", "kill", "-9"])  # Port 5000 freigeben

In [ ]:
# Launch MLflow UI on port 5000
subprocess.Popen(["mlflow", "ui", "--backend-store-uri", mlflow_storage_path]) 

<Popen: returncode: None args: ['mlflow', 'ui', '--backend-store-uri', '/Use...>

INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)
INFO:     Started parent process [26918]
INFO:     Started server process [26923]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Started server process [26922]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Started server process [26920]
INFO:     Waiting for application startup.
INFO:     Started server process [26921]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Application startup complete.


In [6]:
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
conf.get_default().auth_token = input()
port=5000
public_url = ngrok.connect(port).public_url
print(f' * ngrok tunnel (ULR for MLFLow UI) \"{public_url}\" ')

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
 * ngrok tunnel (ULR for MLFLow UI) "https://5403e8b87b01.ngrok-free.app" 


INFO:     127.0.0.1:52478 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:52478 - "GET /ajax-api/2.0/mlflow/experiments/search?max_results=25&order_by=last_update_time+DESC HTTP/1.1" 200 OK


## 2. Import Libraries and DataFrame

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from darts import TimeSeries
import requests
import xgboost as xgb
from xgboost import XGBRegressor
from xgboost import plot_importance, plot_tree
from xgboost.sklearn import XGBModel 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score , mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV 


In [8]:
df_boost = pd.read_pickle("/Users/lukasfichtner/Documents/Guayas_project_week3/xgboost_dataset.pkl")

In [25]:
df_boost.head()

,date,store_nbr,item_nbr,id,unit_sales,lag_1,lag_7,lag_30,rolling_mean_7,sales_change_7d,dcoilwtico,day,month,year,day_of_week,is_weekend
0,2013-01-09,24,96995,302230,2.0,0.0,0.0,0.0,0.0,0.0,93.08,9,1,2013,2,0
1,2013-01-10,24,96995,0,0.0,2.0,0.0,0.0,0.0,0.0,93.81,10,1,2013,3,0
2,2013-01-11,24,96995,0,0.0,0.0,0.0,0.0,0.0,0.0,93.60,11,1,2013,4,0
3,2013-01-12,24,96995,419989,2.0,0.0,0.0,0.0,0.0,0.0,93.60,12,1,2013,5,1
4,2013-01-13,24,96995,0,0.0,2.0,0.0,0.0,0.0,0.0,93.60,13,1,2013,6,1


INFO:     127.0.0.1:52897 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:52900 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=5c81ae8e05f744d794d51bbfaff608e1 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52897 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=a1e34909b93a46fb93ead1848d5d0b8d HTTP/1.1" 200 OK
INFO:     127.0.0.1:52901 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=b29876393f284328a10bfbb1d52712d9 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52902 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=18137aa70ae1422cbdd72b89b30c034f HTTP/1.1" 200 OK
INFO:     127.0.0.1:52899 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=36852558ad374d3b92268592122b928b HTTP/1.1" 200 OK
INFO:     127.0.0.1:52898 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=3d4fe2979d234f0298dae31e5eec6532 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52901 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=8ae4f5cbfc5e418f88a863f053c20cf2 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52900 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=89ebd8

## 3. XGBoost Model

### 1. Train-Test Split

In [10]:

#Splitting the train and test data by a specified date
split_date = '2014-01-01'
train = df_boost[df_boost['date'] < split_date]
test = df_boost[df_boost['date'] > split_date]

In [11]:
# Define target variable (unit_sales) and features
X_train = train.drop('unit_sales', axis=1)
X_train = X_train.drop('date', axis=1)
y_train = train['unit_sales']

X_test = test.drop('unit_sales', axis=1)
X_test = X_test.drop('date', axis=1)
y_test = test['unit_sales']

In [12]:
# Ensure features are scaled for XGBoost (optional, but can help with convergence)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### 2. Train XGBoost Model

In [38]:
# Initialize the XGBoost regressor
xgboost_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    max_depth=5,
    eta=0.1,
    enable_categorical=True
)

xgboost_model.fit(X_train, y_train) 

INFO:     127.0.0.1:53512 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:53517 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=b29876393f284328a10bfbb1d52712d9 HTTP/1.1" 200 OK
INFO:     127.0.0.1:53512 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=ab65122ac1154fe1a831a6112e268b4a HTTP/1.1" 200 OK
INFO:     127.0.0.1:53514 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=3d4fe2979d234f0298dae31e5eec6532 HTTP/1.1" 200 OK
INFO:     127.0.0.1:53515 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=36852558ad374d3b92268592122b928b HTTP/1.1" 200 OK
INFO:     127.0.0.1:53513 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=a1e34909b93a46fb93ead1848d5d0b8d HTTP/1.1" 200 OK
INFO:     127.0.0.1:53516 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=5c81ae8e05f744d794d51bbfaff608e1 HTTP/1.1" 200 OK
INFO:     127.0.0.1:53517 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=18137aa70ae1422cbdd72b89b30c034f HTTP/1.1" 200 OK
INFO:     127.0.0.1:53512 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=4d326e

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,True
,eval_metric,None


In [39]:
# Make predictions on the test set
y_pred = xgboost_model.predict(X_test)

### 3. Create metrics function

In [40]:
def forecast_metrics(y_true, y_pred):
    #converts input data into flattened NumPy arrays
    y_true = np.asarray(y_true, dtype=float).flatten()
    y_pred = np.asarray(y_pred, dtype=float).flatten()

    # basic errors
    errors       = y_true - y_pred
    abs_errors   = np.abs(errors)
    pct_errors   = abs_errors / np.where(y_true == 0, np.nan, y_true)   # avoid ÷0

    # core statistics
    mae   = abs_errors.mean()
    bias  = errors.mean()
    rmse  = np.sqrt((errors ** 2).mean())

    # MAD of the ACTUAL series (dispersion of demand itself)
    mad   = np.abs(y_true - y_true.mean()).mean()

    # Relative MAD = MAE divided by MAD  (how big the forecast error is vs. “typical” variation)
    rmad  = mae / mad if mad else np.nan

    # Mean Absolute Percentage Error
    mape  = np.nanmean(pct_errors) * 100   # expressed in %

    return {
        "MAE" : mae,
        "Bias": bias,
        "MAD" : mad,
        "rMAD": rmad,
        "MAPE": mape,
        "RMSE": rmse
    }


metrics = forecast_metrics(y_test, y_pred)
for k, v in metrics.items():
    print(f"{k}: {v:,.2f}")

MAE: 0.90
Bias: -0.21
MAD: 5.29
rMAD: 0.17
MAPE: 31.89
RMSE: 4.90


In [41]:
with mlflow.start_run(run_name="xgb-normal"):
    xgboost_model.fit(X_train, y_train)
    y_pred = xgboost_model.predict(X_test)
    metrics = forecast_metrics(y_test, y_pred)

    mlflow.log_param("n_estimators", 200)
    mlflow.log_param("max_depth", 5)
    mlflow.log_param("learning_rate", 0.1)
    mlflow.log_param("enable_categorical", True)
    mlflow.log_metric("MAE", metrics["MAE"])
    mlflow.log_metric("Bias", metrics["Bias"])
    mlflow.log_metric("MAD", metrics["MAD"])
    mlflow.log_metric("rMAD", metrics["rMAD"])
    mlflow.log_metric("MAPE", metrics["MAPE"])
    mlflow.log_metric("RMSE", metrics["RMSE"])

    mlflow.sklearn.log_model(xgboost_model, artifact_path="model")

2025/09/02 09:06:13 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/02 09:06:15 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/18/sn83f4h148vgxw6xg2nvxl7r0000gn/T/tmpq7j8_0ri/model/model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.7.1', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 
2025/09/02 09:06:15 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [42]:
# Quick plot saver
def save_forecast_plot(y_true, Y_pred, path, title="Forecast vs. Actual"):
    plt.figure(figsize=(10, 4))
    plt.plot(y_true, label='Actual', lw=2)
    plt.plot(y_pred, label='Predicted', lw=2)
    plt.title(title)
    plt.legend(); plt.tight_layout();plt.savefig(path);plt.close()

# one-stop logger for a single candidate
def log_candidate(run_name, params, y_true, y_pred, fig_name, model=None):
    with mlflow.start_run(run_name=run_name, nested=True):
        mlflow.log_params(params)
        mlflow.log_metrics(forecast_metrics(y_true, y_pred))
        save_forecast_plot(y_true, y_pred, fig_name, title=run_name)
        mlflow.log_artifact(fig_name)

        if model is None:
            return
        elif isinstance(model, XGBModel):  # covers XGBRegressor/Classifier
            mlflow.xgboost.log_model(model, name="model")
        else:
            # fallback: skip or raise
            pass
        

### 4. Hyperparameter Tuning for XGBoost Model improvement

In [59]:
# Initializing the model
xgboost_model = xgb.XGBRegressor(objective='reg:squarederror')

# Define the parameter grid
param_grid = {
    'eta': [0.01, 0.05, 0.1, 0.3], # Extend learning rates
    'max_depth': [ 3, 5, 6, 7, 9], # Wider depth range
    'subsample': [ 0.7, 0.8, 0.9, 1.0], # More subsampling options
    'colsample_bytree': [ 0.7, 0.8, 0.9, 1.0], # More features samplig
    'n_estimators': [50, 100, 200, 300, 400], # Wider tree count range
    'min_child_weight': [1, 3, 5, 7], # better Control of overitting
    'gamma': [0, 0.1, 0.2, 0.3, 0.4], # Minimum loss reduction
    'lambda': [0.5, 1, 1.5, 2], # Regularization
    'alpha': [0, 0.1, 0.5, 1] # Regularization
}

# Time series split for cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Randomized search with cross-validation
random_search = RandomizedSearchCV(estimator=xgboost_model,
                                   param_distributions=param_grid,
                                   cv=tscv,
                                   n_iter=100,
                                   scoring='neg_mean_squared_error', # this is the MSE metric
                                   verbose=2
                                  )

# Fit the model and search for the best hyperparameters
random_search.fit(X_train, y_train)

# Get best hyperparameters
best_params = random_search.best_params_
print("Best Parameters for XGBoost:", best_params)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END alpha=0.5, colsample_bytree=0.8, eta=0.1, gamma=0.1, lambda=1, max_depth=5, min_child_weight=7, n_estimators=300, subsample=0.8; total time=   2.2s
[CV] END alpha=0.5, colsample_bytree=0.8, eta=0.1, gamma=0.1, lambda=1, max_depth=5, min_child_weight=7, n_estimators=300, subsample=0.8; total time=   3.8s
[CV] END alpha=0.5, colsample_bytree=0.8, eta=0.1, gamma=0.1, lambda=1, max_depth=5, min_child_weight=7, n_estimators=300, subsample=0.8; total time=   5.6s
[CV] END alpha=0.5, colsample_bytree=0.8, eta=0.1, gamma=0.1, lambda=1, max_depth=5, min_child_weight=7, n_estimators=300, subsample=0.8; total time=   7.5s
[CV] END alpha=0.5, colsample_bytree=0.8, eta=0.1, gamma=0.1, lambda=1, max_depth=5, min_child_weight=7, n_estimators=300, subsample=0.8; total time=   9.1s
[CV] END alpha=0, colsample_bytree=0.7, eta=0.01, gamma=0.4, lambda=1.5, max_depth=9, min_child_weight=3, n_estimators=200, subsample=0.8; total time=  

In [61]:
def forecast_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    bias = np.mean(y_pred - y_true)
    
    # Calculate MAPE (Mean Absolute Percentage Error)
    # Avoid division by zero
    mask = y_true != 0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100 if np.any(mask) else np.nan
    
    # Calculate MAD (Median Absolute Deviation) and rMAD (relative MAD)
    mad = np.median(np.abs(y_pred - y_true))
    rmad = mad / np.median(np.abs(y_true)) if np.median(np.abs(y_true)) != 0 else np.nan
    
    return {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'R2': r2,
        'Bias': bias,
        'MAPE': mape,
        'MAD': mad,
        'rMAD': rmad
    }

In [62]:

# Start a PARENT MLflow run to group all candidate models from this search
with mlflow.start_run(run_name="XGB_hyperparam_search"):

    # Log all CV results, not individual test evaluations
    mlflow.log_params(best_params)
    #mlflow.log_metric("best_cv_score", -random_search.best_score_)  # Convert back from negative MSE

    # ONLY NOW train the final model on full training data
    final_model = XGBRegressor(**best_params, random_state=42)
    final_model.fit(X_train, y_train)

    # FINALLY evaluate on test set (only once!)
    y_pred = final_model.predict(X_test)

    # Log the final model evaluation
    log_candidate(
        run_name="xgb_final_model",
        params={"model_type": "XGB", **best_params},
        y_true=y_test,
        y_pred=y_pred,
        fig_name="xgb_final_forecast.png",
        model=final_model
    )



INFO:     127.0.0.1:55076 - "GET /ajax-api/2.0/mlflow/model-versions/search?filter=tags.%60mlflow.prompt.is_prompt%60+%3D+%27true%27+AND+tags.%60mlflow.prompt.run_ids%60+ILIKE+%22%25f2a9134f59344ab784bb6423882e41a3%25%22 HTTP/1.1" 200 OK
INFO:     127.0.0.1:55076 - "GET /ajax-api/2.0/mlflow/model-versions/search?filter=tags.%60mlflow.prompt.is_prompt%60+%3D+%27true%27+AND+tags.%60mlflow.prompt.run_ids%60+ILIKE+%22%25f2a9134f59344ab784bb6423882e41a3%25%22 HTTP/1.1" 200 OK
INFO:     127.0.0.1:55076 - "GET /ajax-api/2.0/mlflow/experiments/search?max_results=25&order_by=last_update_time+DESC HTTP/1.1" 200 OK
INFO:     127.0.0.1:55076 - "POST /ajax-api/2.0/mlflow/experiments/search-datasets HTTP/1.1" 200 OK
INFO:     127.0.0.1:55078 - "GET /ajax-api/2.0/mlflow/gateway-proxy?gateway_path=api%2F2.0%2Fendpoints%2F HTTP/1.1" 200 OK
INFO:     127.0.0.1:55080 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:55079 - "POST /ajax-api/2.0/mlflow/logged-models/search HTTP/

/Users/lukasfichtner/Documents/Guayas_project_week3/.venv/lib/python3.13/site-packages/xgboost/sklearn.py:1028: UserWarning: [11:17:21] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/09/02 11:17:23 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/18/sn83f4h148vgxw6xg2nvxl7r0000gn/T/tmpd7i9ta81/model, flavor: xgboost). Fall back to return ['xgboost==3.0.4']. Set logging level to DEBUG to see the full traceback. 
2025/09/02 11:17:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


INFO:     127.0.0.1:55089 - "POST /ajax-api/2.0/mlflow/runs/delete HTTP/1.1" 200 OK
INFO:     127.0.0.1:55089 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=496b3c4a883f4c8da252703cf96cb34a HTTP/1.1" 200 OK
INFO:     127.0.0.1:55089 - "POST /ajax-api/2.0/mlflow/runs/search HTTP/1.1" 200 OK
INFO:     127.0.0.1:55089 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=496b3c4a883f4c8da252703cf96cb34a HTTP/1.1" 200 OK
INFO:     127.0.0.1:55091 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=1a48865e291f4f73afe74902a7f0ae3f HTTP/1.1" 200 OK
INFO:     127.0.0.1:55094 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=74bb18fc13204299b0fafe61cf4a46fd HTTP/1.1" 200 OK
INFO:     127.0.0.1:55090 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=47164badbc0848f8a6fdf6516f432782 HTTP/1.1" 200 OK
INFO:     127.0.0.1:55093 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=251f1bd524f843639cf403017d746d8c HTTP/1.1" 200 OK
INFO:     127.0.0.1:55089 - "GET /ajax-api/2.0/mlflow/runs/get?run_id=ab65122ac1154fe1a831a6112e268b4a HTTP/1.1"